In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/celp/Data_explore.ipynb
/kaggle/input/celp/train_data.csv
/kaggle/input/celp/test_data.csv
/kaggle/input/celp/my_first_submission.csv


In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
#import os
#import re
import nltk
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from collections import Counter

2024-02-23 09:26:33.474061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 09:26:33.474157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 09:26:33.754035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def remove_punct(text):

#     new_text = word_tokenize(text)
#     new_text = list(filter(lambda token: token not in string.punctuation, new_text))
#     text = " ".join([word for word in new_text])
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_text = tokenizer.tokenize(text)
    text = " ".join([word for word in new_text])

    return text.strip()

In [6]:
# 1. Importer les bibliothèques nécessaires
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

In [7]:
# 2. Préparer les données
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        item = {key: val.squeeze() for key, val in inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [8]:
# Charger votre dataset
df = pd.read_csv('/kaggle/input/celp/train_data.csv') # Mettre à jour avec le chemin correct
df["text"] = df["text"].apply(lambda text :remove_punct(text))
texts = df['text'].tolist()
labels = df['level'].tolist()

In [9]:
df.head(30)

,Id,text,level
0,0,My friend Meg was going out to work like every...,3
1,1,Hello I m sorry but i m on holiday when you ca...,0
2,2,Name MaryAge 62Hair color blondeEyes greenDesc...,0
3,3,hi my name is laila i live in paris from franc...,0
4,4,A pink sweater 10 A black dress with a white b...,0
5,5,Hi Granny How are you Look at my photos On the...,1
6,6,Mark an area of about 8 meters by 3 Fill with ...,2
7,7,I believe that in 50 years the world will be m...,2
8,8,Yes Jeff the monkeys are more friendly than li...,1
9,9,I have been in london I have never seen people...,1


In [10]:
all_words = df['text'].str.split(expand=True).stack()
word_counts = Counter(all_words)
most_common = word_counts.most_common(200)

In [11]:
most_common = word_counts.most_common(200)

In [12]:
# Étape 1: Tokenisation et comptage global
all_words = df['text'].str.split(expand=True).stack()
word_counts = Counter(all_words)

# Sélectionner les 200 mots les plus communs
most_common_words = [word for word, count in word_counts.most_common(200)]

# Étape 2: Calculer la fréquence par niveau pour chaque mot
# Initialiser un dictionnaire pour stocker les comptages
word_distribution = {word: [0, 0, 0, 0, 0, 0] for word in most_common_words}

# Itérer sur chaque ligne pour mettre à jour la fréquence de chaque mot par niveau
for _, row in df.iterrows():
    text = row['text']
    level = row['level']
    # Compter les mots dans le texte actuel
    text_word_counts = Counter(text.split())
    # Mettre à jour la fréquence des mots dans le dictionnaire
    for word in most_common_words:
        if word in text_word_counts:
            word_distribution[word][level] += text_word_counts[word]

# Étape 3: Créer un DataFrame à partir du dictionnaire
df_word_distribution = pd.DataFrame.from_dict(word_distribution, orient='index', columns=[0, 1, 2, 3, 4, 5])
df_word_distribution.reset_index(inplace=True)
df_word_distribution.rename(columns={'index': 'Word'}, inplace=True)

print(df_word_distribution)

      Word      0      1      2     3     4    5
0        I  16279  19707  14542  7025  1189   78
1      the   9402  15206  14273  9482  2768  355
2      and  11944  12401  10081  5559  1650  169
3       to   5378   8451  12933  8563  2148  218
4        a   8141   9589  10339  5975  1523  179
..     ...    ...    ...    ...   ...   ...  ...
195  party    326    554     55    39    18    2
196    eat    530    318     82    24    21    1
197   come    289    362    218    77    20    2
198  never    169    401    214   142    39    1
199   best    112    248    294   216    89    6

[200 rows x 7 columns]


In [13]:
# Diviser les données
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.15)

In [19]:
# Initialiser le tokenizer
import transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v2-xlarge')

OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/google/gemma-7b and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [15]:
# Créer les datasets
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
# 3. Charger DistilBERT pré-entraîné
model_name='google/gemma-7b'
num_labels=6
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# 4. Définir le DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [18]:
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel, TrainingArguments, Trainer

# Assuming your BERT model is already trained and loaded into the 'model' variable
# train_dataset and val_dataset should be DataLoader instances containing your training and validation data

# Training BERT model
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=6,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    warmup_steps=200,                
    weight_decay=0.02,               
    logging_dir='./logs',            
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids, p.predictions.argmax(-1))},
)

trainer.train()




wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/deberta/modeling_deberta.py", line 1202, in forward
    outputs = self.deberta(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  

In [28]:
# 6. Évaluer le modèle
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


AttributeError: 'NotebookTrainingTracker' object has no attribute 'value'

In [ ]:
# Obtenir les résultats de la prédiction
predictions = trainer.predict(val_dataset)

# Les logits des prédictions sont dans `predictions.predictions`
logits = predictions.predictions

# Convertir les logits en indices de classe prédite
y_pred = logits.argmax(axis=-1)

In [ ]:
cm = confusion_matrix(val_dataset.labels, y_pred)
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2, 3, 4, 5], yticklabels=[0, 1, 2, 3, 4, 5])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Obtenir les résultats de la prédiction
predictions = trainer.predict(train_dataset)

# Les logits des prédictions sont dans `predictions.predictions`
logits = predictions.predictions

# Convertir les logits en indices de classe prédite
y_pred_train = logits.argmax(axis=-1)

cm = confusion_matrix(train_dataset.labels, y_pred_train)

 
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2, 3, 4, 5], yticklabels=[0, 1, 2, 3, 4, 5])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader

# Charger les données de test
test_df = pd.read_csv("/kaggle/input/celp/test_data.csv")  # Mettre à jour avec le chemin correct
test_df["text"] = test_df["text"].apply(lambda text :remove_punct(text))
test_texts = test_df['text'].tolist()
test_ids = test_df['Id'].tolist()

# Préparer le dataset de test
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.tokenizer = tokenizer
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        item = {key: val.squeeze() for key, val in inputs.items()}
        return item

test_dataset = TestDataset(test_texts, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

# Faire des prédictions
model.eval()  # Mettre le modèle en mode évaluation
predictions = []

with torch.no_grad():
    for batch in test_loader:
        outputs = model(**{k: v.to(model.device) for k, v in batch.items()})
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Préparer le fichier de soumission
submission_df = pd.DataFrame({'Id': test_ids, 'level': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Fichier de soumission créé avec succès !")